In [35]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split, cross_val_score,KFold,RepeatedKFold,GridSearchCV
from scipy.stats import pearsonr, ttest_ind, levene
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [36]:
data1 = pd.read_csv('t-test_data.csv',header='infer')
#data2 = data1.drop(['Unnamed: 0'],axis=1)
#data2['TYPE'] = data2['TYPE'].replace(['HCC','Adjacent'],[0,1])
data1

,Unnamed: 0,TYPE,A1BG,A2M,NAT1,NAT2,SERPINA3,AADAC,AAMP,AARS,...,LOC100287227,NCRNA00173,FAM7A2,LOC100288778,LOC100289341,ZNF605,LOC100302401,LOC100302650,TSNAX-DISC1,LOC100303728
0,0,2.676428,1.160184,1.056793,1.703307,1.508964,1.204877,0.719424,0.645937,-0.148447,...,-0.691617,-1.068722,-0.532467,0.660801,0.596977,0.155580,0.223263,0.078438,-0.412758,1.065182
1,1,-0.373632,0.894721,0.600501,-0.730966,-1.351296,0.781387,0.472509,0.387510,-0.044547,...,0.914283,-0.405976,-0.532467,0.250100,-0.167223,-0.658198,0.389484,-0.286649,-0.412758,-0.373713
2,2,-0.373632,-0.437019,-0.312084,-0.834753,0.176744,-1.360681,-1.231205,0.309982,0.163253,...,0.116871,-1.068722,-0.532467,-0.012293,-0.178140,2.197422,-0.519191,-0.860357,-0.412758,-0.746760
3,3,-0.373632,0.204517,1.466878,0.656004,0.866930,0.392368,0.600905,0.232455,0.215203,...,-0.414737,-1.068722,3.479419,-1.232987,1.219254,-0.184728,-0.463784,-0.674088,-0.412758,0.329747
4,4,-0.373632,0.000995,-0.976307,-0.070504,-0.407507,0.156002,-0.169470,1.938069,0.977135,...,0.172247,0.803086,-0.532467,-0.445810,-0.320063,-1.057689,0.954635,-0.271747,1.126294,0.127236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,-0.373632,-0.335258,-1.998632,0.608828,0.976076,-0.287185,1.025599,0.309982,0.509586,...,1.833522,-0.074604,-0.532467,-1.495379,2.059874,-0.303095,1.165182,-1.307402,-0.412758,0.862671
396,396,-0.373632,-0.658238,0.357915,-0.410170,-1.598479,-0.242866,-0.095396,-0.439454,-0.893063,...,0.714930,-0.065648,3.377422,-0.126376,-0.603909,1.709155,0.544623,-0.361156,-0.412758,0.009992
397,397,-0.373632,-0.127312,-1.588547,-0.797012,0.122171,-0.400444,0.674979,-0.723723,0.145936,...,0.526652,-0.173120,-0.532467,0.010524,1.983454,0.436703,1.852229,-0.793300,-0.412758,1.118474
398,398,-0.373632,-0.056522,0.046019,0.401254,-0.160324,-0.587567,0.092260,-0.258556,1.479318,...,0.194397,0.068693,1.626471,-0.673977,1.579520,-0.258707,0.256507,1.017232,2.010806,0.649501


In [16]:
data_1 = data2[data2['TYPE'] == 0]
data_2 = data2[data2['TYPE'] == 1]
rows_1,__ = data_1.shape
rows_2,__ = data_2.shape
data = pd.concat([data_1,data_2])
data = shuffle(data)
X = data[data.columns[2:]]
y = data['TYPE']
colNames = X.columns
X = X.astype(np.float64)
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X)
X.columns = colNames

In [17]:
# t-test for feature selection
index = []
for colName in data.columns[2:]:
    if levene(data_1[colName],data_2[colName])[1] > 0.05:
        if ttest_ind(data_1[colName],data_2[colName])[1] < 0.05:
            index.append(colName)
    else:
        if ttest_ind(data_1[colName],data_2[colName],equal_var = False)[1] < 0.05:
            index.append(colName)
print(len(colName))

/home/ubuntu/anaconda3/lib/python3.9/site-packages/scipy/stats/_morestats.py:2643: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom


12


In [18]:
# to select the 'positive' features
if 'TYPE' not in index:index = ['TYPE']+index
data_1 = data_1[index]
data_2 = data_2[index]
data = pd.concat([data_1,data_2])
data = shuffle(data)
data.index = range(len(data))#re-label after mixure
X = data[data.columns[2:]]
y = data['TYPE']
X = X.apply(pd.to_numeric,errors = 'ignore') # transform the type of the data 
colNames = X.columns # to read the feature's name
X = X.astype(np.float64)
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X)
X.columns = colNames

In [31]:
# lasso for further feature selection
alphas = np.logspace(-3,1,30)
model_lassoCV = LassoCV(alphas = alphas, cv = 10, max_iter = 100000).fit(X,y)
print(model_lassoCV.alpha_)

0.001


In [32]:
coef = pd.Series(model_lassoCV.coef_,index = X.columns)

In [34]:
index = coef[coef != 0].index
X = X[index]
X.head()
print(coef[coef !=0])


ACTC1          -0.002835
ADCYAP1R1       0.009394
ALDH1B1         0.003627
AQP8           -0.006129
CACNA1F        -0.008536
                  ...   
C10orf131      -0.005011
EFCAB10        -0.000573
LOC100134868   -0.001462
C5orf52         0.000410
FAM7A2         -0.002775
Length: 180, dtype: float64
